In [ ]:
import random
from tqdm import tqdm
from MonBigTool import IS_levenshtein_distance_and_operations
from MonBigTool import MonBigTool,MASKmodel
from collections import Counter

mASKmodel = MASKmodel()
monBigTool = MonBigTool()

In [ ]:
MASK = monBigTool.getMASK()
WINDOW = 3

LOSS_PScore = 0
LOSS_PError = 0
LOSS_PMiss = 0
LOSS_PTrue = 0
LOSS_PTrue_errFix = 0
LOSS_PTrue_succFix = 0



COUNT = 0
LENMASK = len(MASK)
for ii in tqdm(range(LENMASK)):
    ii = random.choice(range(LENMASK))
    i = MASK[ii]
    COUNT += 1
    # i = random.choice()
    inode,tsen,fsen,sen,freq,pairs = monBigTool.Decode(i)

    # ########################################
    # # 得到词对 信息  ########################
    # for g in pairs:
    #     if not g[0].isalpha():
    #         continue


    #     # te = monBigTool.letterSim(g[0],g[1])
    #     # if te < 0.8:
    #         # print(g[0],g[1],te)
    # ########################################


    temp = mASKmodel.tomask(fsen,WINDOW,False)
    SecondaryTreatment = []
    for i in range(len(fsen)):
        if not fsen[i].isalpha():
            continue
        if fsen[i] in temp[i] and len(fsen[i]) > 2:
            continue
        SecondaryTreatment.append(i)

    print('假----',' '.join(fsen),'----')
    print('真----',' '.join(tsen),'----')
    print('MASK----',' '.join(sen),'----')
    print('怀疑----',SecondaryTreatment)


    for i in SecondaryTreatment:
        TTEMP = []
        Alternate = []
        temp = monBigTool.FuzzySearch(fsen[i])

        print(fsen[i],':  :',temp)
        for new in temp:
            fsen[i] = new
            AAA = mASKmodel.AttFix(fsen,WINDOW,i)
            TTEMP += AAA              

        if len(TTEMP) == 0:
            continue

        counter = Counter(TTEMP)
        TTEMP, count = counter.most_common(1)[0]

        print('结果:',TTEMP)
        if fsen[i] in TTEMP:
            continue
    
        if i not in inode:
            LOSS_PError += 1
            print('*ERR'*20)
            print('----',' '.join(fsen),'----')
            print('----',' '.join(tsen),'----')
            print('----',' '.join(sen),'----')
            print(fsen[i],temp[i])
            print(fsen[i],TTEMP)
        elif i in inode:
            LOSS_PTrue += 1
            inode.remove(i)
            if tsen[i] == TTEMP:
                LOSS_PTrue_succFix += 1
            else:
                LOSS_PTrue_errFix += 1
    
    LOSS_PMiss += len(inode)
    for i in inode:
        print('@MISS'*20)
        print('----',' '.join(fsen),'----')
        print('----',' '.join(tsen),'----')
        print('----',' '.join(sen),'----')
        print(fsen[i],temp[i])
        print(fsen[i],TTEMP)

    # if COUNT % 50 == 0:
    print('-----------------')
    print('PError:',LOSS_PError)
    print('PMiss:',LOSS_PMiss)
    print('PTrue:',LOSS_PTrue)
    print('PTrue_succFix:',LOSS_PTrue_succFix)
    print('PTrue_errFix:',LOSS_PTrue_errFix)
    


In [ ]:
print('-----------------')
print('PError:',LOSS_PError)
print('PMiss:',LOSS_PMiss)
print('PTrue:',LOSS_PTrue)
print('PTrue_succFix:',LOSS_PTrue_succFix)
print('PTrue_errFix:',LOSS_PTrue_errFix)

# PError: 9
# PMiss: 104
# PTrue: 86
# PTrue_succFix: 78
# PTrue_errFix: 8

# PError: 5
# PMiss: 107
# PTrue: 82
# PTrue_succFix: 78
# PTrue_errFix: 4

In [ ]:
for ii in range(len(MASK)):
    i = MASK[ii]
    inode,tsen,fsen,sen,freq,pairs = monBigTool.Decode(i)
    if fsen[0] == 'энэ':
        print(ii)


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-uncased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-uncased')

# 词嵌入
input_text = "тан"
              
encoded_input = tokenizer.encode(input_text, return_tensors='pt')
outputs = model(encoded_input)
last_hidden_states = outputs[0]
# 解码 last_hidden_states

# 选择每个位置最可能的 token ID
predicted_ids = torch.argmax(last_hidden_states[0], dim=1)


# 解码 predicted_ids
for i in predicted_ids:
    print(tokenizer.decode(i))

print(tokenizer.decode(predicted_ids.tolist()))
last_hidden_states.shape
stelist = ['aa1','aa','dsada1sd','']
ssd=[1,2,3,4]
[ssd[i] for i in range(len(stelist)) if stelist[i].isalpha() or stelist[i]=='']


In [ ]:
from collections import Counter

TTEMP = [1,2,3,3,12,2,4,45,3,534,5,43]

# 使用 Counter 计算每个元素的出现次数
counter = Counter(TTEMP)

# 找出出现次数最多的元素
most_common_element, count = counter.most_common(1)[0]

print(most_common_element)